In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sigmatrader.data.apikey import APIKey
from binance.spot import Spot
from dataclasses import dataclass

In [2]:
%load_ext autoreload
%autoreload 2
pd.options.mode.copy_on_write = True

In [3]:
import requests
import json

In [4]:
url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest"
parameters = {
  'start':'1',
  'limit':'100',
  'sort':'market_cap',
  'sort_dir':'desc'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': APIKey.coinmarketcap_key,
}

response = requests.get(url, headers=headers, params=parameters)
data = response.json()

In [5]:
data = response.json()["data"]
full_mkt_cap_df = pd.DataFrame(data)
# display(mkt_cap_df)

get_mkt_cap = np.vectorize(lambda x: x["USD"]["market_cap"])
get_dominance = np.vectorize(lambda x: x["USD"]["market_cap_dominance"])

full_mkt_cap_df["market_cap"] = get_mkt_cap(full_mkt_cap_df["quote"])
full_mkt_cap_df["dominance_pct"] = get_dominance(full_mkt_cap_df["quote"])
display(full_mkt_cap_df)

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,infinite_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote,market_cap,dominance_pct
0,1,Bitcoin,BTC,bitcoin,10924,2010-07-13T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.965450e+07,1.965450e+07,False,None,1,NaN,NaN,NaN,2024-03-14T23:41:00.000Z,"{'USD': {'price': 71656.98789378506, 'volume_2...",1.408382e+12,51.8608
1,1027,Ethereum,ETH,ethereum,8563,2015-08-07T00:00:00.000Z,"[pos, smart-contracts, ethereum-ecosystem, coi...",NaN,1.200858e+08,1.200858e+08,True,None,2,NaN,NaN,NaN,2024-03-14T23:41:00.000Z,"{'USD': {'price': 3887.407727084719, 'volume_2...",4.668226e+11,17.1743
2,825,Tether USDt,USDT,tether,78607,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,1.031300e+11,1.067955e+11,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",3,NaN,NaN,NaN,2024-03-14T23:35:00.000Z,"{'USD': {'price': 0.9997090407307992, 'volume_...",1.031000e+11,3.7981
3,1839,BNB,BNB,bnb,2103,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",NaN,1.495391e+08,1.495391e+08,False,None,4,NaN,NaN,NaN,2024-03-14T23:41:00.000Z,"{'USD': {'price': 605.7588636772595, 'volume_2...",9.058466e+10,3.3356
4,5426,Solana,SOL,solana,639,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,4.435908e+08,5.718484e+08,True,None,5,NaN,NaN,NaN,2024-03-14T23:35:00.000Z,"{'USD': {'price': 174.90899625577524, 'volume_...",7.758802e+10,2.8582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1376,Neo,NEO,neo,319,2016-09-08T00:00:00.000Z,"[platform, enterprise-solutions, smart-contracts]",1.000000e+08,7.053883e+07,1.000000e+08,False,None,96,NaN,NaN,NaN,2024-03-14T23:41:00.000Z,"{'USD': {'price': 17.329672296232857, 'volume_...",1.222415e+09,0.0450
96,7186,PancakeSwap,CAKE,pancakeswap,1351,2020-09-25T00:00:00.000Z,"[collectibles-nfts, decentralized-exchange-dex...",4.500000e+08,2.474540e+08,3.887262e+08,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",97,NaN,NaN,-0.188714,2024-03-14T23:30:00.000Z,"{'USD': {'price': 4.793928142985334, 'volume_2...",1.186277e+09,0.0437
97,4846,Kava,KAVA,kava,156,2019-10-25T00:00:00.000Z,"[cosmos-ecosystem, defi, binance-launchpad, bi...",1.082861e+09,1.082861e+09,1.082861e+09,True,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",98,1.082861e+09,1.168060e+09,NaN,2024-03-14T23:41:00.000Z,"{'USD': {'price': 1.0786794960949768, 'volume_...",1.168060e+09,0.0430
98,11156,dYdX (ethDYDX),ETHDYDX,dydx-ethdydx,285,2021-08-03T00:00:00.000Z,"[cosmos-ecosystem, decentralized-exchange-dex-...",NaN,2.956164e+08,5.360440e+08,False,"{'id': 1027, 'name': 'Ethereum', 'symbol': 'ET...",98,3.778217e+08,1.487951e+09,NaN,2024-03-14T23:23:00.000Z,"{'USD': {'price': 3.9382368490708464, 'volume_...",1.164208e+09,0.0428


# Coin selection
## Market cap
## Volume
## Filters
- Not a stablecoin
- Date of inception < 2021-01-01


In [6]:
cols_n_types = [
    ("symbol", str),
    ("name", str),
    ("date_added", datetime),
    ("market_cap", float),
    ("dominance_pct", float),
    ("quote", object),
    ("circulating_supply", float),
    ("total_supply", float),
    ("infinite_supply", float),
    ("tags", str),
]
mkt_cap_df = full_mkt_cap_df[[c for c, _ in cols_n_types]]

# fix types
def fix_types(df, columns_and_types):
    for c, t in columns_and_types:
        df[c] = df.loc[:, c].astype(t) if t is not datetime else pd.to_datetime(df.loc[:, c], utc=True)


fix_types(mkt_cap_df, cols_n_types)

In [7]:
# filter for stablecoins
stablecoin_filter = "tags.str.contains('stablecoin') == False"
# filter for nfts
nfts_filter = "tags.str.contains('collectibles-nfts') == False"
# filter for date of inception
inception_date_filter = "date_added < '2021-01-01 00:00:00+00:00'"
# inception_date_filter = "date_added < '2021-01-01 00:00:00'"

def chain_filters(df, filters):
    # return df.query(" and ".join(filters)).reset_index(drop=True)
    return df.query(" and ".join(filters))
    
filtered_df = chain_filters(
    mkt_cap_df, 
    [
        stablecoin_filter, 
        nfts_filter,
        inception_date_filter,
    ]
)
display(filtered_df)

,symbol,name,date_added,market_cap,dominance_pct,quote,circulating_supply,total_supply,infinite_supply,tags
0,BTC,Bitcoin,2010-07-13 00:00:00+00:00,1.408382e+12,51.8608,"{'USD': {'price': 71656.98789378506, 'volume_2...",1.965450e+07,1.965450e+07,0.0,"['mineable', 'pow', 'sha-256', 'store-of-value..."
1,ETH,Ethereum,2015-08-07 00:00:00+00:00,4.668226e+11,17.1743,"{'USD': {'price': 3887.407727084719, 'volume_2...",1.200858e+08,1.200858e+08,1.0,"['pos', 'smart-contracts', 'ethereum-ecosystem..."
3,BNB,BNB,2017-07-25 00:00:00+00:00,9.058466e+10,3.3356,"{'USD': {'price': 605.7588636772595, 'volume_2...",1.495391e+08,1.495391e+08,0.0,"['marketplace', 'centralized-exchange', 'payme..."
4,SOL,Solana,2020-04-10 00:00:00+00:00,7.758802e+10,2.8582,"{'USD': {'price': 174.90899625577524, 'volume_...",4.435908e+08,5.718484e+08,1.0,"['pos', 'platform', 'solana-ecosystem', 'cms-h..."
5,XRP,XRP,2013-08-04 00:00:00+00:00,3.674649e+10,1.3519,"{'USD': {'price': 0.6705541273904305, 'volume_...",5.480019e+10,9.998780e+10,0.0,"['medium-of-exchange', 'enterprise-solutions',..."
7,ADA,Cardano,2017-10-01 00:00:00+00:00,2.674752e+10,0.9854,"{'USD': {'price': 0.7522222149061217, 'volume_...",3.555801e+10,3.670029e+10,0.0,"['dpos', 'pos', 'platform', 'research', 'smart..."
8,DOGE,Dogecoin,2013-12-15 00:00:00+00:00,2.552476e+10,0.9399,"{'USD': {'price': 0.17791098723427798, 'volume...",1.434693e+11,1.434693e+11,1.0,"['mineable', 'pow', 'scrypt', 'medium-of-excha..."
9,AVAX,Avalanche,2020-07-13 00:00:00+00:00,2.042337e+10,0.7514,"{'USD': {'price': 54.12871579478629, 'volume_2...",3.773112e+08,4.360008e+08,0.0,"['defi', 'smart-contracts', 'three-arrows-capi..."
10,SHIB,Shiba Inu,2020-08-01 00:00:00+00:00,1.903169e+10,0.7013,"{'USD': {'price': 3.229599654747498e-05, 'volu...",5.892894e+14,5.895341e+14,0.0,"['memes', 'ethereum-ecosystem', 'doggone-dogge..."
11,DOT,Polkadot,2020-08-19 00:00:00+00:00,1.477500e+10,0.5441,"{'USD': {'price': 11.454921315237467, 'volume_...",1.289839e+09,1.422099e+09,1.0,"['substrate', 'polkadot', 'binance-chain', 'po..."


In [8]:
client = Spot(api_key=APIKey.binance_api_key, api_secret=APIKey.binance_pvt_key)
KLINES_COLUMNS = [ 
    "opentime",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "closetime",
    "quotevolume",
    "numtrades",
    "takerbuyvolume",
    "takerbuyquotevolume",
    "ignore",
]

USDT_SYMBOL = "USDT"

exchange_info = client.exchange_info(permissions=["SPOT"])

spot_symbols = exchange_info["symbols"]

base_assets = [s["baseAsset"] for s in spot_symbols if s["quoteAsset"] == USDT_SYMBOL]
filtered_df = filtered_df[filtered_df["symbol"].isin(base_assets)]
symbols = filtered_df["symbol"].values

In [9]:
symbols

array(['BTC', 'ETH', 'BNB', 'SOL', 'XRP', 'ADA', 'DOGE', 'AVAX', 'SHIB',
       'DOT', 'LINK', 'MATIC', 'TRX', 'NEAR', 'BCH', 'UNI', 'LTC', 'FIL',
       'ATOM', 'ETC', 'INJ', 'HBAR', 'XLM', 'GRT', 'RUNE', 'VET', 'LDO',
       'XMR', 'MKR', 'AR', 'ALGO', 'FTM', 'AAVE', 'EGLD', 'QNT', 'AGIX',
       'BTT', 'SNX', 'HNT', 'EOS', 'IOTA', 'NEO', 'KAVA', 'ZRX'],
      dtype=object)

In [10]:
start_dt = datetime(2021,1,1)
end_dt = datetime(2023,12,31)
early_dt = datetime(2000, 1, 1, 0, 0)

dt_to_time = lambda d: int(datetime.timestamp(d)) * 1000


frames = [pd.DataFrame([], columns=["pair", "avg_volume"])]
total_volumes = []
for s in symbols:
    pair = f"{s}{USDT_SYMBOL}"
    klines_data = client.klines(symbol=pair, interval="1M", startTime=dt_to_time(start_dt), endTime=dt_to_time(end_dt))
    klines_df = pd.DataFrame(
        klines_data,
        columns=KLINES_COLUMNS
    )
    
    avg_volume = klines_df["quotevolume"].astype(float).sum()
    
    inception_klines = client.klines(symbol=pair, interval="1h", startTime=dt_to_time(early_dt),limit=1)
    inception_klines_df = pd.DataFrame(
            inception_klines,
            columns=KLINES_COLUMNS
        )
    exchange_dt_added = datetime.fromtimestamp(inception_klines_df.iloc[0,:]["opentime"]/1000)
    
    frames.append(pd.DataFrame([(s, pair, avg_volume, exchange_dt_added)], columns=["symbol", "pair", "avg_volume", "exchange_date_added"]))

volume_df = pd.concat(frames)
display(volume_df)

/var/folders/tm/3zwjdtkd273_yy4slcmrlmgr0000gn/T/ipykernel_40228/1777872918.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  volume_df = pd.concat(frames)


,pair,avg_volume,symbol,exchange_date_added
0,BTCUSDT,3.274104e+12,BTC,2017-08-17 05:00:00
0,ETHUSDT,1.479512e+12,ETH,2017-08-17 05:00:00
0,BNBUSDT,4.598127e+11,BNB,2017-11-06 03:00:00
0,SOLUSDT,2.622869e+11,SOL,2020-08-11 07:00:00
0,XRPUSDT,3.895913e+11,XRP,2018-05-04 09:00:00
0,ADAUSDT,2.738304e+11,ADA,2018-04-17 05:00:00
0,DOGEUSDT,4.185695e+11,DOGE,2019-07-05 13:00:00
0,AVAXUSDT,1.272372e+11,AVAX,2020-09-22 07:00:00
0,SHIBUSDT,2.172411e+11,SHIB,2021-05-10 12:00:00
0,DOTUSDT,1.975508e+11,DOT,2020-08-19 00:00:00


In [11]:
# join with volume data
full_df = filtered_df.set_index("symbol").join(volume_df.set_index("symbol"))
display(full_df)
full_df = full_df.sort_values("avg_volume", ascending=False)
full_df.isna().any()

,name,date_added,market_cap,dominance_pct,quote,circulating_supply,total_supply,infinite_supply,tags,pair,avg_volume,exchange_date_added
symbol,,,,,,,,,,,,
BTC,Bitcoin,2010-07-13 00:00:00+00:00,1.408382e+12,51.8608,"{'USD': {'price': 71656.98789378506, 'volume_2...",1.965450e+07,1.965450e+07,0.0,"['mineable', 'pow', 'sha-256', 'store-of-value...",BTCUSDT,3.274104e+12,2017-08-17 05:00:00
ETH,Ethereum,2015-08-07 00:00:00+00:00,4.668226e+11,17.1743,"{'USD': {'price': 3887.407727084719, 'volume_2...",1.200858e+08,1.200858e+08,1.0,"['pos', 'smart-contracts', 'ethereum-ecosystem...",ETHUSDT,1.479512e+12,2017-08-17 05:00:00
BNB,BNB,2017-07-25 00:00:00+00:00,9.058466e+10,3.3356,"{'USD': {'price': 605.7588636772595, 'volume_2...",1.495391e+08,1.495391e+08,0.0,"['marketplace', 'centralized-exchange', 'payme...",BNBUSDT,4.598127e+11,2017-11-06 03:00:00
SOL,Solana,2020-04-10 00:00:00+00:00,7.758802e+10,2.8582,"{'USD': {'price': 174.90899625577524, 'volume_...",4.435908e+08,5.718484e+08,1.0,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",SOLUSDT,2.622869e+11,2020-08-11 07:00:00
XRP,XRP,2013-08-04 00:00:00+00:00,3.674649e+10,1.3519,"{'USD': {'price': 0.6705541273904305, 'volume_...",5.480019e+10,9.998780e+10,0.0,"['medium-of-exchange', 'enterprise-solutions',...",XRPUSDT,3.895913e+11,2018-05-04 09:00:00
ADA,Cardano,2017-10-01 00:00:00+00:00,2.674752e+10,0.9854,"{'USD': {'price': 0.7522222149061217, 'volume_...",3.555801e+10,3.670029e+10,0.0,"['dpos', 'pos', 'platform', 'research', 'smart...",ADAUSDT,2.738304e+11,2018-04-17 05:00:00
DOGE,Dogecoin,2013-12-15 00:00:00+00:00,2.552476e+10,0.9399,"{'USD': {'price': 0.17791098723427798, 'volume...",1.434693e+11,1.434693e+11,1.0,"['mineable', 'pow', 'scrypt', 'medium-of-excha...",DOGEUSDT,4.185695e+11,2019-07-05 13:00:00
AVAX,Avalanche,2020-07-13 00:00:00+00:00,2.042337e+10,0.7514,"{'USD': {'price': 54.12871579478629, 'volume_2...",3.773112e+08,4.360008e+08,0.0,"['defi', 'smart-contracts', 'three-arrows-capi...",AVAXUSDT,1.272372e+11,2020-09-22 07:00:00
SHIB,Shiba Inu,2020-08-01 00:00:00+00:00,1.903169e+10,0.7013,"{'USD': {'price': 3.229599654747498e-05, 'volu...",5.892894e+14,5.895341e+14,0.0,"['memes', 'ethereum-ecosystem', 'doggone-dogge...",SHIBUSDT,2.172411e+11,2021-05-10 12:00:00


name                   False
date_added             False
market_cap             False
dominance_pct          False
quote                  False
circulating_supply     False
total_supply           False
infinite_supply        False
tags                   False
pair                   False
avg_volume             False
exchange_date_added    False
dtype: bool

In [12]:
fix_types(full_df, [("exchange_date_added", datetime)])
# # filter for exchange date added
exchange_date_filter = "exchange_date_added < '2021-01-01 00:00:00+00:00'"
full_df = chain_filters(full_df, [exchange_date_filter])

In [13]:

full_df = full_df.sort_values("avg_volume", ascending=False).reset_index()
display(full_df)
n = len(full_df)

hi_df = full_df[:n//3]
mid_df =full_df[n//3:(n//3)*2]
lo_df = full_df[(n//3)*2:]

,symbol,name,date_added,market_cap,dominance_pct,quote,circulating_supply,total_supply,infinite_supply,tags,pair,avg_volume,exchange_date_added
0,BTC,Bitcoin,2010-07-13 00:00:00+00:00,1.408382e+12,51.8608,"{'USD': {'price': 71656.98789378506, 'volume_2...",1.965450e+07,1.965450e+07,0.0,"['mineable', 'pow', 'sha-256', 'store-of-value...",BTCUSDT,3.274104e+12,2017-08-17 05:00:00+00:00
1,ETH,Ethereum,2015-08-07 00:00:00+00:00,4.668226e+11,17.1743,"{'USD': {'price': 3887.407727084719, 'volume_2...",1.200858e+08,1.200858e+08,1.0,"['pos', 'smart-contracts', 'ethereum-ecosystem...",ETHUSDT,1.479512e+12,2017-08-17 05:00:00+00:00
2,BNB,BNB,2017-07-25 00:00:00+00:00,9.058466e+10,3.3356,"{'USD': {'price': 605.7588636772595, 'volume_2...",1.495391e+08,1.495391e+08,0.0,"['marketplace', 'centralized-exchange', 'payme...",BNBUSDT,4.598127e+11,2017-11-06 03:00:00+00:00
3,DOGE,Dogecoin,2013-12-15 00:00:00+00:00,2.552476e+10,0.9399,"{'USD': {'price': 0.17791098723427798, 'volume...",1.434693e+11,1.434693e+11,1.0,"['mineable', 'pow', 'scrypt', 'medium-of-excha...",DOGEUSDT,4.185695e+11,2019-07-05 13:00:00+00:00
4,XRP,XRP,2013-08-04 00:00:00+00:00,3.674649e+10,1.3519,"{'USD': {'price': 0.6705541273904305, 'volume_...",5.480019e+10,9.998780e+10,0.0,"['medium-of-exchange', 'enterprise-solutions',...",XRPUSDT,3.895913e+11,2018-05-04 09:00:00+00:00
5,ADA,Cardano,2017-10-01 00:00:00+00:00,2.674752e+10,0.9854,"{'USD': {'price': 0.7522222149061217, 'volume_...",3.555801e+10,3.670029e+10,0.0,"['dpos', 'pos', 'platform', 'research', 'smart...",ADAUSDT,2.738304e+11,2018-04-17 05:00:00+00:00
6,SOL,Solana,2020-04-10 00:00:00+00:00,7.758802e+10,2.8582,"{'USD': {'price': 174.90899625577524, 'volume_...",4.435908e+08,5.718484e+08,1.0,"['pos', 'platform', 'solana-ecosystem', 'cms-h...",SOLUSDT,2.622869e+11,2020-08-11 07:00:00+00:00
7,DOT,Polkadot,2020-08-19 00:00:00+00:00,1.477500e+10,0.5441,"{'USD': {'price': 11.454921315237467, 'volume_...",1.289839e+09,1.422099e+09,1.0,"['substrate', 'polkadot', 'binance-chain', 'po...",DOTUSDT,1.975508e+11,2020-08-19 00:00:00+00:00
8,MATIC,Polygon,2019-04-28 00:00:00+00:00,1.210801e+10,0.4456,"{'USD': {'price': 1.222202259161951, 'volume_2...",9.906716e+09,1.000000e+10,0.0,"['pos', 'platform', 'enterprise-solutions', 'z...",MATICUSDT,1.802481e+11,2019-04-26 16:00:00+00:00
9,AVAX,Avalanche,2020-07-13 00:00:00+00:00,2.042337e+10,0.7514,"{'USD': {'price': 54.12871579478629, 'volume_2...",3.773112e+08,4.360008e+08,0.0,"['defi', 'smart-contracts', 'three-arrows-capi...",AVAXUSDT,1.272372e+11,2020-09-22 07:00:00+00:00


In [25]:
@dataclass
class Coin:
    name: str
    symbol: str
    pair: str
    avg_volume: float
    market_cap: float
    dominance_pct: float
    exchange_date_added: datetime
    tags: list[str]

    def __str__(self):
        return f"{self.name} ({self.symbol}) \n market_cap: {self.market_cap} \n avg_volume: {self.avg_volume} \n dominance: {self.dominance_pct:.2f}% \n exchange_date_added: {self.exchange_date_added}\n tags: {self.tags}"


In [27]:
row = hi_df.iloc[0,:]
hi_coin = Coin(row["name"], row["symbol"], row["pair"], row["avg_volume"], row["market_cap"], row["dominance_pct"], row["exchange_date_added"], row["tags"])
print(hi_coin)

Bitcoin (BTC) 
 market_cap: 1408382268558.3984 
 avg_volume: 3274103846094.9565 
 dominance: 51.86% 
 exchange_date_added: 2017-08-17 05:00:00+00:00
 tags: ['mineable', 'pow', 'sha-256', 'store-of-value', 'state-channel', 'coinbase-ventures-portfolio', 'three-arrows-capital-portfolio', 'polychain-capital-portfolio', 'binance-labs-portfolio', 'blockchain-capital-portfolio', 'boostvc-portfolio', 'cms-holdings-portfolio', 'dcg-portfolio', 'dragonfly-capital-portfolio', 'electric-capital-portfolio', 'fabric-ventures-portfolio', 'framework-ventures-portfolio', 'galaxy-digital-portfolio', 'huobi-capital-portfolio', 'alameda-research-portfolio', 'a16z-portfolio', '1confirmation-portfolio', 'winklevoss-capital-portfolio', 'usv-portfolio', 'placeholder-ventures-portfolio', 'pantera-capital-portfolio', 'multicoin-capital-portfolio', 'paradigm-portfolio', 'bitcoin-ecosystem', 'ftx-bankruptcy-estate']


In [29]:
row = mid_df.iloc[0,:]
mid_coin = Coin(row["name"], row["symbol"], row["pair"], row["avg_volume"], row["market_cap"], row["dominance_pct"], row["exchange_date_added"], row["tags"])
print(mid_coin)

TRON (TRX) 
 market_cap: 11332408899.054058 
 avg_volume: 110405595953.26697 
 dominance: 0.42% 
 exchange_date_added: 2018-06-11 12:00:00+00:00
 tags: ['media', 'payments', 'tron-ecosystem', 'layer-1', 'dwf-labs-portfolio', 'sec-security-token', 'alleged-sec-securities']


In [30]:
row = lo_df.iloc[0,:]
lo_coin = Coin(row["name"], row["symbol"], row["pair"], row["avg_volume"], row["market_cap"], row["dominance_pct"], row["exchange_date_added"], row["tags"])
print(lo_coin)

Algorand (ALGO) 
 market_cap: 2522666753.747291 
 avg_volume: 38808697786.78066 
 dominance: 0.09% 
 exchange_date_added: 2019-06-22 01:00:00+00:00
 tags: ['pos', 'platform', 'research', 'smart-contracts', 'arrington-xrp-capital-portfolio', 'kenetic-capital-portfolio', 'usv-portfolio', 'multicoin-capital-portfolio', 'exnetwork-capital-portfolio', 'layer-1', 'dwf-labs-portfolio', 'sec-security-token', 'alleged-sec-securities']
